<a href="https://colab.research.google.com/github/nullvoid-ky/introduction-to-machine-learning-and-deep-learning/blob/main/stdscaler_smote_rf_7_feat_selected.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ===== Setup & Installs (Kaggle usually has most of these; safe to re-run) =====
!pip -q install kagglehub shap lightgbm xgboost

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import List
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.datasets import make_classification
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.metrics import accuracy_score, classification_report
import warnings
warnings.filterwarnings('ignore')


In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("utkarshx27/american-companies-bankruptcy-prediction-dataset")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'american-companies-bankruptcy-prediction-dataset' dataset.
Path to dataset files: /kaggle/input/american-companies-bankruptcy-prediction-dataset


In [3]:
from kagglehub import KaggleDatasetAdapter, load_dataset

# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
# Set the CSV file path **inside** the dataset (adjust if needed)
# Explore the dataset directory printed below to confirm the file name.
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
file_path = "/kaggle/input/american-companies-bankruptcy-prediction-dataset/american_bankruptcy.csv"

df = pd.read_csv(file_path)

print("Loaded shape:", df.shape)
print("Columns:\n", list(df.columns))
df.head()

Loaded shape: (78682, 21)
Columns:
 ['company_name', 'status_label', 'year', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12', 'X13', 'X14', 'X15', 'X16', 'X17', 'X18']


,company_name,status_label,year,X1,X2,X3,X4,X5,X6,X7,...,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18
0,C_1,alive,1999,511.267,833.107,18.373,89.031,336.018,35.163,128.348,...,1024.333,740.998,180.447,70.658,191.226,163.816,201.026,1024.333,401.483,935.302
1,C_1,alive,2000,485.856,713.811,18.577,64.367,320.590,18.531,115.187,...,874.255,701.854,179.987,45.790,160.444,125.392,204.065,874.255,361.642,809.888
2,C_1,alive,2001,436.656,526.477,22.496,27.207,286.588,-58.939,77.528,...,638.721,710.199,217.699,4.711,112.244,150.464,139.603,638.721,399.964,611.514
3,C_1,alive,2002,396.412,496.747,27.172,30.745,259.954,-12.410,66.322,...,606.337,686.621,164.658,3.573,109.590,203.575,124.106,606.337,391.633,575.592
4,C_1,alive,2003,432.204,523.302,26.680,47.491,247.245,3.504,104.661,...,651.958,709.292,248.666,20.811,128.656,131.261,131.884,651.958,407.608,604.467


In [4]:

FEATURES_OLD = ["X8","X17","X3","X11","X10","X1","X6"]
# FEATURES = ["X1","X2","X3","X4","X5","X6","X7","X8","X9","X11","X12","X13","X14","X15","X16","X17","X18","year"]
FEATURES = FEATURES_OLD
TARGET   = "status_label"
COMPANY  = "company_name"   # ถ้าไม่มีคอลัมน์นี้ โค้ดจะ fallback อัตโนมัติ


In [5]:

# 0) ตรวจว่าคอลัมน์ครบไหม
missing = [c for c in FEATURES+[TARGET] if c not in df.columns]
if missing:
    raise ValueError(f"❌ Missing columns: {missing}")

# 1) ฟังก์ชัน normalize label ให้เป็น 0/1 แบบทนทาน
def normalize_status(x):
    if pd.isna(x):
        return np.nan
    t = str(x).strip().lower()
    # ตัวเลขที่มาเป็นสตริง หรือ float 0.0/1.0
    if t in {"0","1"}:
        return int(t)
    try:
        # กรณีเป็น 0.0/1.0 จริง ๆ
        f = float(t)
        if f in (0.0, 1.0):
            return int(f)
    except:
        pass
    # แม็พคำยอดฮิต
    direct = {
        "alive": 0, "non-bankrupt": 0, "nonbankrupt": 0, "healthy": 0, "normal": 0,
        "failed": 1, "fail": 1, "bankrupt": 1, "bankruptcy": 1, "went_bankrupt": 1,
        "yes": 1, "y": 1, "true": 1,
        "no": 0, "n": 0, "false": 0
    }
    if t in direct:
        return direct[t]
    # สุดท้าย ถ้าระบุไม่ถูก ให้คืน NaN เพื่อตรวจสอบ
    return np.nan

y_norm = df[TARGET].apply(normalize_status)

# 2) เช็คค่าที่แปลงไม่ได้ (จะเป็น NaN)
bad_mask = y_norm.isna()
if bad_mask.any():
    print("⚠️ พบ label ที่ไม่รู้จัก (ตัวอย่าง top 20):")
    print(df.loc[bad_mask, TARGET].value_counts().head(20))
    # ทางเลือก: ตัดแถวที่ label ไม่ชัดเจนทิ้งไปก่อน
    df = df.loc[~bad_mask].copy()
    y_norm = y_norm.loc[~bad_mask]

# 3) เขียนกลับเป็นตัวเลข 0/1
df[TARGET] = y_norm.astype(int)

In [6]:
# ==============================
# Load your DataFrame (df)
# ==============================
try:
    df  # noqa: F821
    print("✅ Found existing `df`.")
except NameError:
    import pandas as pd
    print("ℹ️ No existing `df` found. Creating a tiny placeholder. Replace with your CSV load.")
    df = pd.DataFrame({
        "X8":[0.1,0.2,0.3,0.4],
        "X17":[1,2,3,4],
        "X3":[5,6,7,8],
        "X11":[0,1,0,1],
        "X15":[10,11,12,13],
        "X1":[2,3,4,5],
        "X6":[9,8,7,6],
        "status_label":["alive","failed","alive","failed"],
    })
print("df shape:", df.shape)


✅ Found existing `df`.
df shape: (78682, 21)


In [7]:
# ==============================
# Feature selection (X, y) + map target
# ==============================
import numpy as np
import pandas as pd

FEATURES = FEATURES
TARGET   = "status_label"

missing = [c for c in FEATURES + [TARGET] if c not in df.columns]
if missing:
    raise ValueError(f"❌ Missing columns in df: {missing}")

# Make sure the target column is integer type
df[TARGET] = df[TARGET].astype(int)

X = df[FEATURES].copy()
y = df[TARGET].copy()

print("✅ X,y ready.")
print("X shape:", X.shape, "| y counts:", dict(pd.Series(y).value_counts()))

✅ X,y ready.
X shape: (78682, 7) | y counts: {0: np.int64(73462), 1: np.int64(5220)}


In [8]:
# ==============================
# Imports
# ==============================
!pip -q install imbalanced-learn
import warnings
warnings.filterwarnings("ignore")


import matplotlib.pyplot as plt
from abc import ABC, abstractmethod
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.decomposition import PCA

from sklearn.metrics import (
    accuracy_score, confusion_matrix, roc_auc_score, f1_score,
    precision_score, recall_score, roc_curve, auc
)

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.metrics import accuracy_score, classification_report

plt.style.use("ggplot")
RANDOM_STATE = 42


In [11]:
# !pip install imbalanced-learn tqdm
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (
    classification_report, f1_score, precision_score, recall_score,
    roc_auc_score, confusion_matrix, log_loss, accuracy_score
)
from sklearn.preprocessing import StandardScaler
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier

# =============================
# PREPARE DATA
# =============================
# Assume df already loaded
X = df[FEATURES].copy()
y = df['status_label'].copy()

# Coerce y to numpy for some metrics that expect array-like
y = np.asarray(y)

# =============================
# DEFINE PIPELINE
# =============================
pipe = ImbPipeline(steps=[
    ('scaler', StandardScaler()),
    ('smote', SMOTE(random_state=42, k_neighbors=4, sampling_strategy='auto')),
    ('clf', RandomForestClassifier(
        n_estimators=30,
        max_depth=None,
        min_samples_leaf=2,
        n_jobs=-1,
        random_state=42
    ))
])

# =============================
# HELPERS
# =============================
def multiclass_specificity(y_true, y_pred, labels=None):
    """
    Macro-averaged specificity across classes (binary reduces to TN/(TN+FP)).
    """
    if labels is None:
        labels = np.unique(y_true)
    cms = confusion_matrix(y_true, y_pred, labels=labels)
    # If binary cms is 2x2; else kxk. Compute TNR per class (one-vs-rest)
    tnr_list = []
    for i, _ in enumerate(labels):
        # One-vs-rest for class i
        TP = cms[i, i]
        FN = cms[i, :].sum() - TP
        FP = cms[:, i].sum() - TP
        TN = cms.sum() - (TP + FN + FP)
        denom = (TN + FP)
        tnr_list.append(TN / denom if denom > 0 else 0.0)
    return float(np.mean(tnr_list))

def safe_roc_auc(y_true, y_proba, labels=None):
    """
    Binary: use column of positive class (last column by default).
    Multiclass: macro OVR.
    """
    if labels is None:
        labels = np.unique(y_true)
    if len(labels) == 2:
        # Try to find positive class by larger label if numeric; else last column
        pos_idx = 1 if y_proba.shape[1] > 1 else 0
        return roc_auc_score(y_true, y_proba[:, pos_idx])
    else:
        return roc_auc_score(y_true, y_proba, multi_class='ovr', average='macro')

# =============================
# STRATIFIED CV WITH PROGRESS BAR
# =============================
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Track per-fold metrics
metrics_rows = []
avg_cm = None

for fold, (train_idx, test_idx) in enumerate(tqdm(skf.split(X, y), total=10, desc="Training SMOTE-RF folds"), start=1):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    pipe.fit(X_train, y_train)

    # Predictions & probabilities
    y_pred = pipe.predict(X_test)
    # Some classifiers might not have predict_proba; RandomForest does.
    y_proba = pipe.predict_proba(X_test)

    # Confusion matrix (aligned to label order)
    labels = np.unique(y)
    cm = confusion_matrix(y_test, y_pred, labels=labels)
    if avg_cm is None:
        avg_cm = cm.astype(float)
    else:
        avg_cm += cm.astype(float)

    # Metrics
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average='macro', zero_division=0)
    rec = recall_score(y_test, y_pred, average='macro', zero_division=0)  # sensitivity
    spec = multiclass_specificity(y_test, y_pred, labels=labels)
    f1m = f1_score(y_test, y_pred, average='macro')
    auc = safe_roc_auc(y_test, y_proba, labels=labels)
    # For log_loss, pass full proba matrix with columns aligned to labels
    # Ensure proba columns correspond to 'labels' order
    clf_classes = pipe.named_steps['clf'].classes_
    # Reindex y_proba columns to match labels order if necessary
    if not np.array_equal(clf_classes, labels):
        # Map columns
        col_map = {c: i for i, c in enumerate(clf_classes)}
        y_proba_aligned = np.column_stack([y_proba[:, col_map[c]] for c in labels])
    else:
        y_proba_aligned = y_proba
    ll = log_loss(y_test, y_proba_aligned, labels=labels)

    metrics_rows.append({
        "fold": fold,
        "Loss (log loss)": ll,
        "Accuracy": acc,
        "Precision": prec,
        "Sensitivity (Recall)": rec,
        "Specificity (TNR)": spec,
        "F1-score": f1m,
        "ROC AUC": auc
    })

    print(f"\n===== Fold {fold} =====")
    print(classification_report(y_test, y_pred, digits=4))
    print(f"Loss (log loss):        {ll:.6f}")
    print(f"Accuracy:               {acc:.6f}")
    print(f"Precision (macro):      {prec:.6f}")
    print(f"Sensitivity (Recall):   {rec:.6f}")
    print(f"Specificity (TNR):      {spec:.6f}")
    print(f"F1-score (macro):       {f1m:.6f}")
    print(f"ROC AUC:                {auc:.6f}")
    print("Confusion Matrix (rows=true, cols=pred, label order={}):".format(list(labels)))
    print(cm)

# =============================
# FINAL SUMMARY
# =============================
print("\n================ FINAL CV SUMMARY ================\n")
df_metrics = pd.DataFrame(metrics_rows).set_index("fold")
summary = df_metrics.agg(['mean', 'std']).T.rename(columns={'mean': 'Mean', 'std': 'Std'})
print(df_metrics.round(6))
print("\n--- Averages (Mean ± Std) ---")
for metric, row in summary.round(6).iterrows():
    print(f"{metric}: {row['Mean']:.6f} ± {row['Std']:.6f}")

# Average confusion matrix across folds
avg_cm /= len(metrics_rows)
print("\n--- Average Confusion Matrix across folds (rows=true, cols=pred) ---")
print(np.round(avg_cm, 2))

# (Optional) If you still want the original F1-only summary:
print("\n================ ORIGINAL F1 SUMMARY ================\n")
print(f"F1 Scores (macro): {np.round(df_metrics['F1-score'].values, 4)}")
print(f"Mean F1: {df_metrics['F1-score'].mean():.4f} ± {df_metrics['F1-score'].std():.4f}")


Training SMOTE-RF folds:  10%|█         | 1/10 [00:11<01:41, 11.26s/it]


===== Fold 1 =====
              precision    recall  f1-score   support

           0     0.9511    0.9248    0.9378     14693
           1     0.2385    0.3314    0.2774      1044

    accuracy                         0.8854     15737
   macro avg     0.5948    0.6281    0.6076     15737
weighted avg     0.9039    0.8854    0.8940     15737

Loss (log loss):        0.328115
Accuracy:               0.885429
Precision (macro):      0.594799
Sensitivity (Recall):   0.628106
Specificity (TNR):      0.628106
F1-score (macro):       0.607569
ROC AUC:                0.763198
Confusion Matrix (rows=true, cols=pred, label order=[np.int64(0), np.int64(1)]):
[[13588  1105]
 [  698   346]]


Training SMOTE-RF folds:  20%|██        | 2/10 [00:22<01:30, 11.27s/it]


===== Fold 2 =====
              precision    recall  f1-score   support

           0     0.9518    0.9192    0.9352     14693
           1     0.2327    0.3448    0.2779      1044

    accuracy                         0.8811     15737
   macro avg     0.5923    0.6320    0.6066     15737
weighted avg     0.9041    0.8811    0.8916     15737

Loss (log loss):        0.344557
Accuracy:               0.881108
Precision (macro):      0.592253
Sensitivity (Recall):   0.632020
Specificity (TNR):      0.632020
F1-score (macro):       0.606553
ROC AUC:                0.751048
Confusion Matrix (rows=true, cols=pred, label order=[np.int64(0), np.int64(1)]):
[[13506  1187]
 [  684   360]]


Training SMOTE-RF folds:  30%|███       | 3/10 [00:33<01:18, 11.22s/it]


===== Fold 3 =====
              precision    recall  f1-score   support

           0     0.9533    0.9220    0.9374     14692
           1     0.2490    0.3640    0.2957      1044

    accuracy                         0.8850     15736
   macro avg     0.6011    0.6430    0.6165     15736
weighted avg     0.9065    0.8850    0.8948     15736

Loss (log loss):        0.348552
Accuracy:               0.884977
Precision (macro):      0.601145
Sensitivity (Recall):   0.642992
Specificity (TNR):      0.642992
F1-score (macro):       0.616547
ROC AUC:                0.760769
Confusion Matrix (rows=true, cols=pred, label order=[np.int64(0), np.int64(1)]):
[[13546  1146]
 [  664   380]]


Training SMOTE-RF folds:  40%|████      | 4/10 [00:44<01:07, 11.18s/it]


===== Fold 4 =====
              precision    recall  f1-score   support

           0     0.9502    0.9232    0.9365     14692
           1     0.2278    0.3190    0.2658      1044

    accuracy                         0.8831     15736
   macro avg     0.5890    0.6211    0.6011     15736
weighted avg     0.9023    0.8831    0.8920     15736

Loss (log loss):        0.326281
Accuracy:               0.883071
Precision (macro):      0.588980
Sensitivity (Recall):   0.621060
Specificity (TNR):      0.621060
F1-score (macro):       0.601120
ROC AUC:                0.748359
Confusion Matrix (rows=true, cols=pred, label order=[np.int64(0), np.int64(1)]):
[[13563  1129]
 [  711   333]]


Training SMOTE-RF folds:  50%|█████     | 5/10 [00:55<00:55, 11.00s/it]


===== Fold 5 =====
              precision    recall  f1-score   support

           0     0.9518    0.9233    0.9373     14692
           1     0.2406    0.3420    0.2824      1044

    accuracy                         0.8847     15736
   macro avg     0.5962    0.6326    0.6099     15736
weighted avg     0.9046    0.8847    0.8939     15736

Loss (log loss):        0.336173
Accuracy:               0.884723
Precision (macro):      0.596181
Sensitivity (Recall):   0.632623
Specificity (TNR):      0.632623
F1-score (macro):       0.609882
ROC AUC:                0.761377
Confusion Matrix (rows=true, cols=pred, label order=[np.int64(0), np.int64(1)]):
[[13565  1127]
 [  687   357]]

================ FINAL CV SUMMARY ================

      Loss (log loss)  Accuracy  Precision  Sensitivity (Recall)  \
fold                                                               
1            0.328115  0.885429   0.594799              0.628106   
2            0.344557  0.881108   0.592253           

In [12]:
# !pip install imbalanced-learn tqdm
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (
    classification_report, f1_score, precision_score, recall_score,
    roc_auc_score, confusion_matrix, log_loss, accuracy_score
)
from sklearn.preprocessing import StandardScaler
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier

# =============================
# PREPARE DATA
# =============================
# Assume df already loaded
X = df[FEATURES].copy()
y = df['status_label'].copy()

# Coerce y to numpy for some metrics that expect array-like
y = np.asarray(y)

# =============================
# DEFINE PIPELINE
# =============================
pipe = ImbPipeline(steps=[
    ('scaler', StandardScaler()),
    ('smote', SMOTE(random_state=42, k_neighbors=4, sampling_strategy='auto')),
    ('clf', RandomForestClassifier(
        n_estimators=120,
        max_depth=None,
        min_samples_leaf=2,
        n_jobs=-1,
        random_state=42
    ))
])

# =============================
# HELPERS
# =============================
def multiclass_specificity(y_true, y_pred, labels=None):
    """
    Macro-averaged specificity across classes (binary reduces to TN/(TN+FP)).
    """
    if labels is None:
        labels = np.unique(y_true)
    cms = confusion_matrix(y_true, y_pred, labels=labels)
    # If binary cms is 2x2; else kxk. Compute TNR per class (one-vs-rest)
    tnr_list = []
    for i, _ in enumerate(labels):
        # One-vs-rest for class i
        TP = cms[i, i]
        FN = cms[i, :].sum() - TP
        FP = cms[:, i].sum() - TP
        TN = cms.sum() - (TP + FN + FP)
        denom = (TN + FP)
        tnr_list.append(TN / denom if denom > 0 else 0.0)
    return float(np.mean(tnr_list))

def safe_roc_auc(y_true, y_proba, labels=None):
    """
    Binary: use column of positive class (last column by default).
    Multiclass: macro OVR.
    """
    if labels is None:
        labels = np.unique(y_true)
    if len(labels) == 2:
        # Try to find positive class by larger label if numeric; else last column
        pos_idx = 1 if y_proba.shape[1] > 1 else 0
        return roc_auc_score(y_true, y_proba[:, pos_idx])
    else:
        return roc_auc_score(y_true, y_proba, multi_class='ovr', average='macro')

# =============================
# STRATIFIED CV WITH PROGRESS BAR
# =============================
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Track per-fold metrics
metrics_rows = []
avg_cm = None

for fold, (train_idx, test_idx) in enumerate(tqdm(skf.split(X, y), total=5, desc="Training SMOTE-RF folds"), start=1):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    pipe.fit(X_train, y_train)

    # Predictions & probabilities
    y_pred = pipe.predict(X_test)
    # Some classifiers might not have predict_proba; RandomForest does.
    y_proba = pipe.predict_proba(X_test)

    # Confusion matrix (aligned to label order)
    labels = np.unique(y)
    cm = confusion_matrix(y_test, y_pred, labels=labels)
    if avg_cm is None:
        avg_cm = cm.astype(float)
    else:
        avg_cm += cm.astype(float)

    # Metrics
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average='macro', zero_division=0)
    rec = recall_score(y_test, y_pred, average='macro', zero_division=0)  # sensitivity
    spec = multiclass_specificity(y_test, y_pred, labels=labels)
    f1m = f1_score(y_test, y_pred, average='macro')
    auc = safe_roc_auc(y_test, y_proba, labels=labels)
    # For log_loss, pass full proba matrix with columns aligned to labels
    # Ensure proba columns correspond to 'labels' order
    clf_classes = pipe.named_steps['clf'].classes_
    # Reindex y_proba columns to match labels order if necessary
    if not np.array_equal(clf_classes, labels):
        # Map columns
        col_map = {c: i for i, c in enumerate(clf_classes)}
        y_proba_aligned = np.column_stack([y_proba[:, col_map[c]] for c in labels])
    else:
        y_proba_aligned = y_proba
    ll = log_loss(y_test, y_proba_aligned, labels=labels)

    metrics_rows.append({
        "fold": fold,
        "Loss (log loss)": ll,
        "Accuracy": acc,
        "Precision": prec,
        "Sensitivity (Recall)": rec,
        "Specificity (TNR)": spec,
        "F1-score": f1m,
        "ROC AUC": auc
    })

    print(f"\n===== Fold {fold} =====")
    print(classification_report(y_test, y_pred, digits=4))
    print(f"Loss (log loss):        {ll:.6f}")
    print(f"Accuracy:               {acc:.6f}")
    print(f"Precision (macro):      {prec:.6f}")
    print(f"Sensitivity (Recall):   {rec:.6f}")
    print(f"Specificity (TNR):      {spec:.6f}")
    print(f"F1-score (macro):       {f1m:.6f}")
    print(f"ROC AUC:                {auc:.6f}")
    print("Confusion Matrix (rows=true, cols=pred, label order={}):".format(list(labels)))
    print(cm)

# =============================
# FINAL SUMMARY
# =============================
print("\n================ FINAL CV SUMMARY ================\n")
df_metrics = pd.DataFrame(metrics_rows).set_index("fold")
summary = df_metrics.agg(['mean', 'std']).T.rename(columns={'mean': 'Mean', 'std': 'Std'})
print(df_metrics.round(6))
print("\n--- Averages (Mean ± Std) ---")
for metric, row in summary.round(6).iterrows():
    print(f"{metric}: {row['Mean']:.6f} ± {row['Std']:.6f}")

# Average confusion matrix across folds
avg_cm /= len(metrics_rows)
print("\n--- Average Confusion Matrix across folds (rows=true, cols=pred) ---")
print(np.round(avg_cm, 2))

# (Optional) If you still want the original F1-only summary:
print("\n================ ORIGINAL F1 SUMMARY ================\n")
print(f"F1 Scores (macro): {np.round(df_metrics['F1-score'].values, 4)}")
print(f"Mean F1: {df_metrics['F1-score'].mean():.4f} ± {df_metrics['F1-score'].std():.4f}")


Training SMOTE-RF folds:  20%|██        | 1/5 [00:43<02:53, 43.42s/it]


===== Fold 1 =====
              precision    recall  f1-score   support

           0     0.9509    0.9281    0.9394     14693
           1     0.2436    0.3257    0.2787      1044

    accuracy                         0.8882     15737
   macro avg     0.5972    0.6269    0.6090     15737
weighted avg     0.9040    0.8882    0.8956     15737

Loss (log loss):        0.309554
Accuracy:               0.888162
Precision (macro):      0.597231
Sensitivity (Recall):   0.626900
Specificity (TNR):      0.626900
F1-score (macro):       0.609035
ROC AUC:                0.772019
Confusion Matrix (rows=true, cols=pred, label order=[np.int64(0), np.int64(1)]):
[[13637  1056]
 [  704   340]]


Training SMOTE-RF folds:  40%|████      | 2/5 [01:27<02:12, 44.07s/it]


===== Fold 2 =====
              precision    recall  f1-score   support

           0     0.9515    0.9245    0.9378     14693
           1     0.2404    0.3362    0.2804      1044

    accuracy                         0.8855     15737
   macro avg     0.5959    0.6304    0.6091     15737
weighted avg     0.9043    0.8855    0.8942     15737

Loss (log loss):        0.315899
Accuracy:               0.885493
Precision (macro):      0.595936
Sensitivity (Recall):   0.630364
Specificity (TNR):      0.630364
F1-score (macro):       0.609075
ROC AUC:                0.761641
Confusion Matrix (rows=true, cols=pred, label order=[np.int64(0), np.int64(1)]):
[[13584  1109]
 [  693   351]]


Training SMOTE-RF folds:  60%|██████    | 3/5 [02:13<01:29, 44.67s/it]


===== Fold 3 =====
              precision    recall  f1-score   support

           0     0.9525    0.9301    0.9412     14692
           1     0.2612    0.3477    0.2983      1044

    accuracy                         0.8915     15736
   macro avg     0.6068    0.6389    0.6197     15736
weighted avg     0.9067    0.8915    0.8985     15736

Loss (log loss):        0.311016
Accuracy:               0.891459
Precision (macro):      0.606841
Sensitivity (Recall):   0.638900
Specificity (TNR):      0.638900
F1-score (macro):       0.619727
ROC AUC:                0.767854
Confusion Matrix (rows=true, cols=pred, label order=[np.int64(0), np.int64(1)]):
[[13665  1027]
 [  681   363]]


Training SMOTE-RF folds:  80%|████████  | 4/5 [02:56<00:43, 43.91s/it]


===== Fold 4 =====
              precision    recall  f1-score   support

           0     0.9497    0.9302    0.9398     14692
           1     0.2377    0.3065    0.2678      1044

    accuracy                         0.8888     15736
   macro avg     0.5937    0.6183    0.6038     15736
weighted avg     0.9025    0.8888    0.8952     15736

Loss (log loss):        0.313005
Accuracy:               0.888790
Precision (macro):      0.593714
Sensitivity (Recall):   0.618340
Specificity (TNR):      0.618340
F1-score (macro):       0.603804
ROC AUC:                0.759753
Confusion Matrix (rows=true, cols=pred, label order=[np.int64(0), np.int64(1)]):
[[13666  1026]
 [  724   320]]


Training SMOTE-RF folds: 100%|██████████| 5/5 [03:38<00:00, 43.75s/it]


===== Fold 5 =====
              precision    recall  f1-score   support

           0     0.9516    0.9300    0.9407     14692
           1     0.2533    0.3343    0.2882      1044

    accuracy                         0.8904     15736
   macro avg     0.6024    0.6321    0.6144     15736
weighted avg     0.9053    0.8904    0.8974     15736

Loss (log loss):        0.311442
Accuracy:               0.890442
Precision (macro):      0.602430
Sensitivity (Recall):   0.632127
Specificity (TNR):      0.632127
F1-score (macro):       0.614423
ROC AUC:                0.769058
Confusion Matrix (rows=true, cols=pred, label order=[np.int64(0), np.int64(1)]):
[[13663  1029]
 [  695   349]]

================ FINAL CV SUMMARY ================

      Loss (log loss)  Accuracy  Precision  Sensitivity (Recall)  \
fold                                                               
1            0.309554  0.888162   0.597231              0.626900   
2            0.315899  0.885493   0.595936           

In [13]:
f1_scores

[]